## 1. Imports 및 클래스 정의

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import concurrent.futures
from sqlalchemy import create_engine



class Position:
    def __init__(self, buy_price, quantity,order,additional_buy_drop_rate,sell_profit_rate):
        self.buy_price = buy_price
        self.quantity = quantity
        self.order = order  # 포지션의 차수
        self.additional_buy_drop_rate = additional_buy_drop_rate  # 추가 매수 드랍 비율 추가
        self.sell_profit_rate = sell_profit_rate  # 매도 수익률 추가



## 2. 롤링 윈도우 계산 함수

In [ ]:
def rolling_window_calculations(df):
    window_size = max(int(len(df) * 2 / 3), 1)  # 윈도우 크기를 정수형으로 설정, 최소값은 1
    window_size = min(window_size, 5 * 252)  # 윈도우 크기가 5년치를 넘지 않도록 설정
    min_periods = min(window_size, 252)  # 최소 기간 설정

    df = df.copy()
    df = df[df['PBR'] != 0]
    df['five_year_high'] = df['고가'].rolling(window=window_size, min_periods=min_periods).max()
    df['five_year_low'] = df['종가'].rolling(window=window_size, min_periods=min_periods).min()
    df['normalized_price'] = (df['종가'] - df['five_year_low']) / (df['five_year_high'] - df['five_year_low']) * 100  # 진입시기결정
    df['five_year_avg_pbr'] = df['PBR'].rolling(window=window_size, min_periods=min_periods).mean()
    df = df.dropna(subset=['five_year_avg_pbr'])

    return df


## 3. 매수 및 매도 조건 체크 함수

In [3]:
def check_buy_sell_conditions(row, positions, capital, investment_per_split, num_splits, buy_threshold, buy_signals, sell_signals,code):
    # ... (중략)
    return positions, capital, None


## 4. 매수 및 매도 수익률 계산 함수

In [ ]:
def calculate_additional_buy_drop_rate(last_buy_price, five_year_low, num_splits):
    return 1 - ((five_year_low / last_buy_price) ** (1 / (num_splits - 1)))

def calculate_sell_profit_rate(buy_profit_rate):
    sell_profit_rate = (1 / (1 - buy_profit_rate)) - 1
    return sell_profit_rate 


## 5. 포트폴리오 가치 업데이트 함수

In [ ]:
def update_portfolio_values(positions, capital):
    portfolio_value = capital + sum(position.quantity * position.buy_price for position in positions)
    return portfolio_value


## 6. 최종 결과 계산 함수

In [ ]:
def calculate_final_results(initial_capital, final_capital, total_days):
    total_profit = final_capital - initial_capital
    return_on_investment = (total_profit / initial_capital) * 100
    total_years = total_days / 365.25

    if final_capital <= 0:
        cagr = -1
    else:
        cagr = ((final_capital / initial_capital) ** (1 / total_years)) - 1

    return total_profit, return_on_investment, cagr 


## 7. 종목 데이터 로드 및 기간 설정 함수

In [ ]:
def get_trading_dates(stock_data):
    all_dates = set()
    for code, data in stock_data.items():
        all_dates.update(data.index)
    return sorted(list(all_dates))

def load_stock_codes_from_db(engine):
    query = "SELECT stock_code FROM stock_codes"
    with engine.connect() as connection:
        result = connection.execute(query)
        stock_codes = [row['stock_code'] for row in result]
    return stock_codes

def load_stock_data_from_db(engine, stock_code):
    query = f"SELECT * FROM stock_data WHERE stock_code = '{stock_code}'"
    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    df['날짜'] = pd.to_datetime(df['날짜'])
    df.set_index('날짜', inplace=True)
    return df

def load_all_stock_data(stock_codes):
    stock_data = {}
    for code in stock_codes:
        df = load_stock_data_from_db(engine, code)
        df = rolling_window_calculations(df)
        stock_data[code] = df
    print(f'{code} done')
    return stock_data

def get_data_for_period(stock_data, start_year=2009, end_year=2023):
    period_data = {}
    for code, data in stock_data.items():
        start_date = pd.Timestamp(year=start_year, month=1, day=1)
        end_date = pd.Timestamp(year=end_year, month=12, day=31)
        period_data[code] = data[(data.index >= start_date) & (data.index <= end_date)]
    return period_data


## 8. 포트폴리오 백테스팅 함수

In [ ]:
def portfolio_backtesting(stock_codes, initial_capital, num_splits, investment_ratio, buy_threshold, stock_data):
    # ... (중략)
    return positions_dict, total_portfolio_value, portfolio_values_over_time, capital_over_time, buy_signals, sell_signals, all_trading_dates, cagr


## 9. 백테스팅 래퍼 및 평균 CAGR 계산 함수

In [ ]:
def backtesting_wrapper(params):
    try:
        _, _, _, _, _, _, _, cagr = portfolio_backtesting(stock_codes, initial_capital, params[0], params[2], params[1], stock_data)
        return params[0], params[1], params[2], cagr
    except Exception as e:
        print(f'Error in backtesting: {e}')
        return params[0], params[1], params[2], None

def check_if_already_calculated(num_splits, buy_threshold, investment_ratio):
    if os.path.exists(results_file):
        existing_results = pd.read_csv(results_file)
        return any((existing_results['num_splits'] == num_splits) & 
                   (existing_results['buy_threshold'] == buy_threshold) &
                   (existing_results['investment_ratio'] == investment_ratio))
    return False

def run_backtesting_for_period(stock_codes, initial_capital, num_splits, investment_ratio, buy_threshold, start_year, end_year, loaded_stock_data):
    period_stock_data = get_data_for_period(loaded_stock_data, start_year, end_year)
    _, total_portfolio_value, _, _, _, _, _, cagr = portfolio_backtesting(
        stock_codes, initial_capital, num_splits, investment_ratio, buy_threshold, loaded_stock_data)
    return total_portfolio_value, cagr

def calculate_average_results(backtesting_results):
    total_values = [result[0] for result in backtesting_results]
    cagr_values = [result[1] for result in backtesting_results]
    average_total_value = sum(total_values) / len(total_values)
    average_cagr = sum(cagr_values) / len(cagr_values)
    return average_total_value, average_cagr

def average_cagr_wrapper(params):
    period_results = []
    for start_year, end_year in time_periods:
        try:
            total_value, cagr = run_backtesting_for_period(
                stock_codes, initial_capital, params[0], params[2], params[1], start_year, end_year, stock_data)
            period_results.append((total_value, cagr))
            print(f'기간 하나 완료{params}')
        except Exception as e:
            print(f'Error in backtesting for period {start_year}-{end_year}: {e}')
            period_results.append((0, None))
    average_results = calculate_average_results(period_results)
    return params[0], params[1], params[2], average_results[0], average_results[1]


## 10. 메인 실행 코드

In [ ]:

initial_capital = 100000000  # 초기 자본
num_splits_options = [10] 
buy_threshold_options = [30, 35, 40, 45, 50, 55, 60]
investment_ratio_options = [0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

time_periods = [(2009, 2014), (2011, 2016), (2013, 2018), (2015, 2020), (2017, 2022), (2018, 2023), (2009, 2016), (2016, 2023)]

combinations = [(n, b, i) for n in num_splits_options for b in buy_threshold_options for i in investment_ratio_options]

# 모든 종목 데이터 로드
engine = create_engine('sqlite:///stock_data.db')
stock_codes = load_stock_codes_from_db(engine)
stock_data = load_all_stock_data(stock_codes, engine)